In [1]:
%cd ..

/home/zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path as op
import time


from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras_tqdm import TQDMNotebookCallback
import numpy as np

from cascading import cascade_net
from data import MaskedUntouched2DSequence
from evaluate import psnr

Using TensorFlow backend.


In [3]:
# paths
train_path = '../singlecoil_val/'
val_path = '../singlecoil_val/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=8)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [5]:
run_params = {
    'n_cascade': 2, 
    'n_convs': 2, 
    'n_filters': 8,
    'noiseless': False,
}

n_epochs = 10
run_id = f'cascadenet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

cascadenet_af4_1566572390


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [7]:
model, run_metadata = cascade_net(lr=1e-3, **run_params)
print(model.summary(line_length=100))

W0823 16:59:51.010439 140429478225664 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 16:59:51.017043 140429478225664 deprecation.py:323] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:59: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0823 16:59:51.046924 140429478225664 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 16:59:51.052393 140429478225664 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W082

Model: "model_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
kspace_input (InputLayer)        (None, 640, None, 1)  0                                            
____________________________________________________________________________________________________
mask_input (InputLayer)          (None, 640, None)     0                                            
____________________________________________________________________________________________________
ifft_simple (Lambda)             (None, 640, None, 1)  0           kspace_input[0][0]               
                                                                   mask_input[0][0]                 
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 640, None, 1)  0           ifft_si

In [8]:
# simple overfit trials

data = train_gen[0]
val_data = val_gen[1]
model.fit(
    x=data[0], 
    y=data[1], 
    validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
    callbacks=[tboard_cback],
    epochs=1,
    verbose=2, 
    shuffle=False,
)

W0823 16:59:52.130730 140429478225664 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0823 16:59:52.131265 140429478225664 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Train on 8 samples, validate on 38 samples
Epoch 1/1
 - 96s - loss: 5.8385e-06 - mean_squared_error: 6.1160e-11 - keras_psnr: 26.8721 - keras_ssim: 0.5493 - val_loss: 9.9581e-06 - val_mean_squared_error: 1.5986e-10 - val_keras_psnr: 20.9585 - val_keras_ssim: 0.3024


In [14]:
%%timeit

fft_inv = FFT2(np.ones(1)).adj_op
for i in range(len(data[0][0])):
    fft_inv(np.squeeze(data[0][0][i]))

91.3 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
from fourier import FFT2

In [10]:
from tensorflow.python.client import timeline
tl = timeline.Timeline(run_metadata.step_stats)
ctf = tl.generate_chrome_trace_format()
with open('timeline.json', 'w') as f:
    f.write(ctf)